# CS838 Project Stage3

Yuying Chen
Maria Kamenetsky

In [1]:
#import modules
import csv
import py_entitymatching as em
import pandas as pd
import numpy as np
import time
import sys

Our full pipeline of training and testing the ML matcher used below was completed in Stage 3 (which can be found in the respective folder). Results of that process were that the logistic regression performed the best. We follow a similar set up as before:

## Read input tables

We first import and set up A and B, which will be used to set up the matcher:

In [42]:
A = em.read_csv_metadata('../../stage3/DATA/movie1_stage3.csv', key='id',encoding='mac_roman')
B = em.read_csv_metadata('../../stage3/DATA/movie2_stage3.csv', key='movieId',encoding='mac_roman')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [43]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A x B: ' + str(len(A)*len(B)))

Number of tuples in A: 5043
Number of tuples in B: 27278
Number of tuples in A x B: 137562954


In [44]:
A.head() 

,id,title,genres,director_name,plot_keywords,actor_3_facebook_likes,cast_total_facebook_likes,content_rating,budget,duration,...,actor_2_facebook_likes,imdb_score,movie_facebook_likes,language,actor_1_facebook_likes,facenumber_in_poster,actor_1_name,num_user_for_reviews,aspect_ratio,num_critic_for_reviews
0,1,avatar,action|adventure|fantasy|sci-fi,james cameron,avatar|future|marine|native|paraplegic,855,4834,pg-13,237000000,178,...,936,7.9,33000,english,1000,0,cch pounder,3054,1.78,723
1,2,pirates of the caribbean: at world's end,action|adventure|fantasy,gore verbinski,goddess|marriage ceremony|marriage proposal|pirate|singapore,1000,48350,pg-13,300000000,169,...,5000,7.1,0,english,40000,0,johnny depp,1238,2.35,302
2,3,spectre,action|adventure|thriller,sam mendes,bomb|espionage|sequel|spy|terrorist,161,11700,pg-13,245000000,148,...,393,6.8,85000,english,11000,1,christoph waltz,994,2.35,602
3,4,the dark knight rises,action|thriller,christopher nolan,deception|imprisonment|lawlessness|police officer|terrorist plot,23000,106759,pg-13,250000000,164,...,23000,8.5,164000,english,27000,0,tom hardy,2701,2.35,813
4,5,star wars: episode vii - the force awakens,documentary,doug walker,.,.,143,.,.,.,...,12,7.1,0,.,131,0,doug walker,.,.,.


In [45]:
A.columns.values

array(['id', 'title', 'genres', 'director_name', 'plot_keywords',
       'actor_3_facebook_likes', 'cast_total_facebook_likes',
       'content_rating', 'budget', 'duration', 'country', 'color', 'year',
       'num_voted_users', 'director_facebook_likes', 'gross',
       'actor_2_name', 'actor_3_name', 'actor_2_facebook_likes',
       'imdb_score', 'movie_facebook_likes', 'language',
       'actor_1_facebook_likes', 'facenumber_in_poster', 'actor_1_name',
       'num_user_for_reviews', 'aspect_ratio', 'num_critic_for_reviews'], dtype=object)

In [46]:
B.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [47]:
B.columns.values

array(['movieId', 'title', 'genres', 'year'], dtype=object)

In [48]:
em.get_key(A), em.get_key(B)

('id', 'movieId')

## Block Tables To Get Candidate Set

In the previous stage, we blocked on attribute 'year' using an overlap blocker since the same movie must have the same year. The output attributes of table A is limited to id, title, year and genres to match the information provided by table B.

Here, we import that file:

In [49]:
# Create overlap blocker
#ob = em.OverlapBlocker()

#start = time.clock()
#C = ob.block_tables(A, B, 'year', 'year',
 #                  l_output_attrs=['id', 'title', 'year', 'genres'],
  #                 r_output_attrs=['movieId', 'title','year', 'genres'],
   #                overlap_size=1, show_progress=True)



In [51]:
#C2 = C.head(1000)
#C2.head()

In [50]:
#em.to_csv_metadata(C2, '../DATA/c2.csv')

In [52]:
#C = em.read_csv_metadata('../DATA/c2.csv',
 #                       key='_id',
  #                      ltable=A, rtable=B,encoding='mac_roman')

In [53]:
C = em.read_csv_metadata('../../stage3/DATA/candidate_pairs.csv',
                        key='_id',
                        ltable=A, rtable=B,encoding='mac_roman')

In [54]:
len(C)

3288743

## Match tuple pairs in candidate set

### Import Pre-Determined Golden Data

Again, we set up the golden data we used in stage 3:

We randomly sampled 500 tuple pairs for labeling. There are 201 matches and 299 mismatches. **gold.csv** is our gold dataset of perfect matches that we have determined prior to this. 

In [55]:
G = em.read_csv_metadata('../../stage3/DATA/gold.csv', 
                         key='_id',
                            ltable=A, rtable=B, 
                        fk_ltable='ltable_id', fk_rtable='rtable_movieId',
                         encoding='mac_roman')
len(G)

Metadata file is not present in the given path; proceeding to read the csv file.


500

In [56]:
G.head(10)

,_id,ltable_id,rtable_movieId,ltable_movie_title,ltable_title_year,ltable_genres,rtable_title,rtable_year,rtable_genres,label
0,2433,894,64716,seven pounds,2008,drama|romance,Seven Pounds,2008,Drama,1
1,4545,2842,4015,"dude, where's my car?",2000,comedy|mystery,"Dude, Where's My Car?",2000,Comedy|Sci-Fi,1
2,6907,3697,104908,insidious: chapter 2,2013,fantasy|horror|thriller,Insidious: Chapter 2,2013,Horror|Thriller,1
3,24168,4384,2891,"happy, texas",1999,comedy|crime|romance,"Happy, Texas",1999,Comedy,1
4,31352,1844,7980,a bridge too far,1977,drama|history|war,"Bridge Too Far, A",1977,Action|Drama|War,1
5,35987,874,2676,instinct,1999,drama|thriller,Instinct,1999,Drama|Thriller,1
6,37526,2582,1599,steel,1997,action|crime|sci-fi,Steel,1997,Action,1
7,50124,1212,1408,the last of the mohicans,1992,action|adventure|drama|romance|war,"Last of the Mohicans, The",1992,Action|Romance|War|Western,1
8,57296,2987,66200,two lovers,2008,drama|romance,Two Lovers,2008,Drama|Romance,1
9,64913,2952,2841,stir of echoes,1999,horror|mystery|thriller,Stir of Echoes,1999,Horror|Mystery|Thriller,1


### Set up from previous (training golden data)

We replicate the same set up of training the model as in stage 3

In [58]:
#set up features
feature_table = em.get_features_for_matching(A,B)

In [59]:
feature_table.feature_name

0        title_title_jac_qgm_3_qgm_3
1    title_title_cos_dlm_dc0_dlm_dc0
2    title_title_jac_dlm_dc0_dlm_dc0
3                    title_title_mel
4               title_title_lev_dist
5                title_title_lev_sim
6                    title_title_nmw
7                     title_title_sw
Name: feature_name, dtype: object

In [60]:
# Select the attrs. to be included in the feature vector table
attrs_from_table = ['ltable_movie_title', 'ltable_title_year',
                    'rtable_title', 'rtable_year']

#convert to feature vectors
H = em.extract_feature_vecs(G,
                           feature_table=feature_table,
                           attrs_after='label',
                           show_progress=True)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


In [61]:
H.head()

,_id,ltable_id,rtable_movieId,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_jac_dlm_dc0_dlm_dc0,title_title_mel,title_title_lev_dist,title_title_lev_sim,title_title_nmw,title_title_sw,label
0,2433,894,64716,0.400000,0.000000,0.0,0.888889,2,0.833333,10.0,10.0,1
1,4545,2842,4015,0.314286,0.000000,0.0,0.853408,4,0.809524,17.0,17.0,1
2,6907,3697,104908,0.571429,0.333333,0.2,0.887037,2,0.900000,18.0,18.0,1
3,24168,4384,2891,0.400000,0.000000,0.0,0.888889,2,0.833333,10.0,10.0,1
4,31352,1844,7980,0.156250,0.000000,0.0,0.665850,8,0.529412,6.0,11.0,1


### Using the LogRegMatcher

From our previous training and testing, the best ML matcher was the logreg matcher. We use it below to predict on the rest of the candidate tuples:

In [62]:
# Select ML matcher from last time
lg = em.LogRegMatcher(name='LogReg', random_state=0)

In [63]:
# Get the attributes to be projected while training
attrs_to_be_excluded = []
attrs_to_be_excluded.extend(['_id', 'ltable_id', 'rtable_movieId', 'label'])
attrs_to_be_excluded

['_id', 'ltable_id', 'rtable_movieId', 'label']

In [64]:
# Train using feature vectors from the labeled data.
lg.fit(table=H, exclude_attrs=attrs_to_be_excluded, target_attr='label')

### Predict matches in the candidate set using trained matcher

In [65]:
C.head()

,_id,ltable_id,rtable_movieId,ltable_title,ltable_year,ltable_genres,rtable_title,rtable_year,rtable_genres
0,0,1026,1,jade,1995,crime|drama|thriller,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
1,1,2053,1,bad boys,1995,action|comedy|crime|drama|thriller,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
2,2,2822,1,clueless,1995,comedy|romance,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
3,3,4662,1,welcome to the dollhouse,1995,comedy|drama,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
4,4,3593,1,the usual suspects,1995,crime|drama|mystery|thriller,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy


In [66]:
# Select the attrs. to be included in the feature vector table
attrs_from_table = ['ltable_title', 'ltable_year',
                    'rtable_title', 'rtable_year']
# Convert the cancidate set to feature vectors using the feature table
L = em.extract_feature_vecs(C, feature_table=feature_table,
                             attrs_before= attrs_from_table,
                             show_progress=True)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 02:18:36


Predict matches in the candidate set using trained matcher and feature vectors:

In [67]:
L.head()

,_id,ltable_id,rtable_movieId,ltable_title,ltable_year,rtable_title,rtable_year,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_jac_dlm_dc0_dlm_dc0,title_title_mel,title_title_lev_dist,title_title_lev_sim,title_title_nmw,title_title_sw
0,0,1026,1,jade,1995,Toy Story,1995,0.0,0.0,0.0,0.000000,9.0,0.000000,-5.0,0.0
1,1,2053,1,bad boys,1995,Toy Story,1995,0.0,0.0,0.0,0.569444,7.0,0.222222,1.0,2.0
2,2,2822,1,clueless,1995,Toy Story,1995,0.0,0.0,0.0,0.000000,9.0,0.000000,-1.0,0.0
3,3,4662,1,welcome to the dollhouse,1995,Toy Story,1995,0.0,0.0,0.0,0.537037,20.0,0.166667,-11.0,3.0
4,4,3593,1,the usual suspects,1995,Toy Story,1995,0.0,0.0,0.0,0.277778,17.0,0.055556,-8.0,1.0


In [68]:
# Get the attributes to be excluded while predicting 
attrs_to_be_excluded = []
attrs_to_be_excluded.extend(['_id', 'ltable_id', 'rtable_movieId'])
attrs_to_be_excluded.extend(attrs_from_table)
attrs_to_be_excluded

['_id',
 'ltable_id',
 'rtable_movieId',
 'ltable_title',
 'ltable_year',
 'rtable_title',
 'rtable_year']

In [69]:
# Predict the matches
predictions = lg.predict(table=L, exclude_attrs=attrs_to_be_excluded,                          
              append=True, target_attr='predicted', inplace=False)

In [70]:
predictions.head()

,_id,ltable_id,rtable_movieId,ltable_title,ltable_year,rtable_title,rtable_year,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_jac_dlm_dc0_dlm_dc0,title_title_mel,title_title_lev_dist,title_title_lev_sim,title_title_nmw,title_title_sw,predicted
0,0,1026,1,jade,1995,Toy Story,1995,0.0,0.0,0.0,0.000000,9.0,0.000000,-5.0,0.0,0
1,1,2053,1,bad boys,1995,Toy Story,1995,0.0,0.0,0.0,0.569444,7.0,0.222222,1.0,2.0,0
2,2,2822,1,clueless,1995,Toy Story,1995,0.0,0.0,0.0,0.000000,9.0,0.000000,-1.0,0.0,0
3,3,4662,1,welcome to the dollhouse,1995,Toy Story,1995,0.0,0.0,0.0,0.537037,20.0,0.166667,-11.0,3.0,0
4,4,3593,1,the usual suspects,1995,Toy Story,1995,0.0,0.0,0.0,0.277778,17.0,0.055556,-8.0,1.0,0


In [71]:
# Get the attributes to be projected out
attrs_proj = []
attrs_proj.extend(['_id', 'ltable_id', 'rtable_movieId'])
attrs_proj.extend(attrs_from_table)
attrs_proj.append('predicted')

# Project the attributes
predictions = predictions[attrs_proj]

In [72]:
predictions.tail()

,_id,ltable_id,rtable_movieId,ltable_title,ltable_year,rtable_title,rtable_year,predicted
3288738,3288738,4691,131262,destiny,2014,Innocence,2014,0
3288739,3288739,4788,131262,rise of the entrepreneur: the search for a better way,2014,Innocence,2014,0
3288740,3288740,4592,131262,a lego brickumentary,2014,Innocence,2014,0
3288741,3288741,3734,131262,still alice,2014,Innocence,2014,0
3288742,3288742,507,131262,pompeii,2014,Innocence,2014,0


Next, we will take only where predicted == 1 and create that to be a separate csv:

In [73]:
predictpos = predictions.loc[predictions['predicted']==1]
predictpos.head(10)

,_id,ltable_id,rtable_movieId,ltable_title,ltable_year,rtable_title,rtable_year,predicted
31,31,1589,1,toy story,1995,Toy Story,1995,1
414,414,1292,6,kiss of death,1995,Heat,1995,1
432,432,3988,7,friday,1995,Sabrina,1995,1
673,673,718,10,goldeneye,1995,GoldenEye,1995,1
734,734,704,11,the american president,1995,"American President, The",1995,1
825,825,3039,12,dead man walking,1995,Dracula: Dead and Loving It,1995,1
963,963,1126,14,nixon,1995,Nixon,1995,1
1031,1031,351,15,cutthroat island,1995,Cutthroat Island,1995,1
1061,1061,921,16,casino,1995,Casino,1995,1
1176,1176,2536,17,sense and sensibility,1995,Sense and Sensibility,1995,1


In [74]:
predictpos.to_csv('pospredictions.csv', sep=',')